# **Technical Indicators for FAANG Securities**
---

### Import Libraries

In [54]:
import os
import requests 
import pandas_datareader as pdr

import functools
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns
import warnings

from sklearn import preprocessing 

from importlib import reload
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227

### Import Ticker Data

In [55]:
ticker = 'FB'
#ticker = 'AAPL'
#ticker = 'AMZN'
#ticker = 'NFLX'
#ticker = 'GOOG'

In [56]:
stock_df = pdr.get_data_yahoo(ticker, '1980')
# stock_df.to_csv('data/'+csv_title+'.csv')
stock_df.to_csv('data/raw_stocks/'+ticker.upper()+'.csv')

In [57]:
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-04-26,187.190002,179.500000,186.630005,180.949997,31747400,180.949997
2022-04-27,181.210007,169.000000,174.429993,174.949997,71062900,174.949997
2022-04-28,208.529999,192.899994,202.919998,205.729996,100890600,205.729996
2022-04-29,212.479996,199.899994,204.460007,200.470001,49137200,200.470001
2022-05-02,210.860001,201.000000,201.164993,209.169998,42513404,209.169998


In [58]:
stock_df.describe()

,High,Low,Open,Close,Volume,Adj Close
count,2505.000000,2505.000000,2505.000000,2505.000000,2.505000e+03,2505.000000
mean,149.557677,145.952990,147.768824,147.794240,3.093101e+07,147.794240
std,90.591243,88.409953,89.481592,89.514438,2.742705e+07,89.514438
min,18.270000,17.549999,18.080000,17.730000,5.913100e+06,17.730000
25%,77.199997,75.360001,76.180000,76.180000,1.585380e+07,76.180000
50%,142.220001,139.729996,141.000000,141.039993,2.249100e+07,141.039993
75%,195.880005,191.470001,194.029999,193.259995,3.595790e+07,193.259995
max,384.329987,378.809998,381.679993,382.179993,5.735764e+08,382.179993


### Check for missing data

In [59]:
print('No Missing Data') if sum(stock_df.isna().sum())==0 else stock_df.isna().sum()

No Missing Data


---
# Techincal Indicator Calculations
---
- Momentum
- Relative Strength Index (RSI)
- Moving Average Convergence/Divergence(MACD)
- Bollinger Bands

> Momentum

In [60]:
def momentum(close, days):
    m = [None for i in range(days)]
    for i in range(len(close) - days):
        end = i + days
        m.append(close[i] - days)
    return m 

> Relative Strength Index (RSI)

In [61]:
def gain(x):
    if x > 0:
        return x
    else:
        return 0 
def loss(x):
    if x < 0:
        return abs(x)
    else:
        return 0 

    
def rsi(stock):
    rsi_list = [None for i in range(14)]
    stock = stock.Change
    
    # Calculate Relative Strength (RS) for first 15 
    avg_gain = sum([i for i in stock[1:15] if i > 0])/14
    avg_loss = sum([abs(i) for i in stock[1:15] if i < 0])/14 
    rs = avg_gain/avg_loss
    # Calculate Relative Strength Index (RSI) for first 15
    rsi = 100 - (100/(1+rs))
    rsi_list.append(rsi)
    
    # Calculate Relative Strength(RS) for 15+ 
    for i in range(15, len(stock)):
        avg_gain = (avg_gain * 13 + gain(stock[i]))/14
        avg_loss = (avg_loss * 13 + loss(stock[i]))/14
        rs = avg_gain/avg_loss
        
        # Calculate Relative Strength Index (RSI) for 15+ 
        rsi = 100 - (100/(1+rs))
        rsi_list.append(rsi)
    
    return rsi_list

> Moving Average Convergence/Divergence (MACD)

In [62]:
def macd(stock):
    exp1 = stock.Close.ewm(span=12, adjust=False).mean()
    exp2 = stock.Close.ewm(span=26, adjust=False).mean()
    ma_con_div = exp1-exp2
    signal = ma_con_div.ewm(span=9, adjust=False).mean()
    return ma_con_div, signal

> Bollinger Bands

In [63]:
def bollinger(stock, window=21):
    roll_mean = stock.Close.rolling(window).mean()
    roll_std = stock.Close.rolling(window).std()
    upper_band = roll_mean + (roll_std*2)
    lower_band = roll_mean - (roll_std*2)
    return upper_band, lower_band

---
# Add Technical Indicators to Dataframe 
---

In [64]:
# gets list of all files and directories
# git add .ipynb_checkpoints directory -> causes issues 
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    if file.endswith('.csv'):
        name = file.lower().split(".")[0]
        stocks[name] = pd.read_csv('data/raw_stocks/'+file)
    
        # Date Feature
        stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
        stocks[name].set_index('Date', inplace = True)
        # Return Feature 
        stocks[name]['Return'] = round(stocks[name]['Close']/stocks[name]['Open']-1, 3)
        # Change Feature: Change previous day's price by absolute value
        stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
        # Volatility Feature
        stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
        # Moving Average: 7 Days
        stocks[name]['MA7'] = stocks[name].Close.rolling(window=7).mean()
        # Moving Average: 21 Days
        stocks[name]['MA21'] = stocks[name].Close.rolling(window=21).mean()
    
        # Momentum
        stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
        # Relative Strength Index (RSI)
        stocks[name]['RSI'] = rsi(stocks[name])
        # Moving Average Convergence/Divergence (MACD)
        stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
        # Bollinger Bands
        stocks[name]['Upper_Band'], stocks[name]['Lower_Band'] = bollinger(stocks[name])
    
        # Saving
        # stocks[name].dropna(inplace=True)
        stocks[name].to_csv('data/stocks/'+name+'.csv')

In [65]:
stocks[ticker.lower()].tail()

,High,Low,Open,Close,Volume,Adj Close,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_Band,Lower_Band
Date,,,,,,,,,,,,,,,,,
2022-04-26,187.190002,179.500000,186.630005,180.949997,31747400,180.949997,-0.030,-6.040009,33.508425,195.517144,213.676668,185.070007,30.175139,-8.459337,-4.333978,246.190278,181.163057
2022-04-27,181.210007,169.000000,174.429993,174.949997,71062900,174.949997,0.003,-6.000000,33.924116,190.400000,211.360477,181.110001,27.810247,-9.831921,-5.433566,247.622016,175.098939
2022-04-28,208.529999,192.899994,202.919998,205.729996,100890600,205.729996,0.014,30.779999,33.204466,188.745714,210.211429,183.990005,49.622506,-8.339878,-6.014829,245.527826,174.895033
2022-04-29,212.479996,199.899994,204.460007,200.470001,49137200,200.470001,-0.020,-5.259995,32.581143,188.752858,208.907620,177.949997,47.008522,-7.495457,-6.310954,243.503333,174.311906
2022-05-02,210.860001,201.000000,201.164993,209.169998,42513404,209.169998,0.040,8.699997,31.849277,191.767142,208.279524,171.949997,51.554203,-6.054439,-6.259651,242.324009,174.235040


In [66]:
new_df = pd.read_csv('data/stocks/'+ticker.lower()+'.csv')
print('No Missing Data') if sum(new_df.isna().sum())==0 else new_df.isna().sum()

Date           0
High           0
Low            0
Open           0
Close          0
Volume         0
Adj Close      0
Return         0
Change         0
Volatility     1
MA7            6
MA21          20
Momentum       3
RSI           14
MACD           0
Signal         0
Upper_Band    20
Lower_Band    20
dtype: int64